< 3월 13일 과제 >  
1. winequality-white.csv
2. 딥러닝 적용해보기
3. 모델 : LinearRegression - 회귀

In [115]:
# Data Load
import torch
import pandas as pd

wine_df = pd.read_csv('winequality-white.csv', sep=';')
wine_df.info() # data 깔끔

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [116]:
# 중복 확인
wine_df.duplicated().sum()   # 937
wine_df.drop_duplicates(inplace=True)

In [117]:
# 피처 지정
wine_df.corr().abs().quality.sort_values(ascending=False)

quality                 1.000000
alcohol                 0.462869
density                 0.337805
chlorides               0.217739
volatile acidity        0.190678
total sulfur dioxide    0.183356
fixed acidity           0.124636
pH                      0.123829
residual sugar          0.117339
sulphates               0.053200
free sulfur dioxide     0.010507
citric acid             0.007065
Name: quality, dtype: float64

In [118]:
# Split
feature = wine_df[['alcohol', 'density']]
target = wine_df['quality']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=11)

# Modeling
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression()
lin_model.fit(X_train, y_train)
print('Test :', lin_model.score(X_test, y_test))

Test : 0.21233367843325224


In [129]:
feature

,alcohol,density
0,8.8,1.00100
1,9.5,0.99400
2,10.1,0.99510
3,9.9,0.99560
6,9.6,0.99490
...,...,...
4893,11.2,0.99114
4894,9.6,0.99490
4895,9.4,0.99254
4896,12.8,0.98869


<hr>

Deep Learning

In [119]:
import torch
import torch.nn as nn               # 인공신경망 층 관련 모듈
import torch.optim as optim         # 인공신경망 관련 함수
import torch.nn.functional as F     # 최적화 모듈

In [120]:
torch.manual_seed(718)      # 랜덤 시드 설정

In [121]:
# Transform train, test data as tensor type
X_train_ten = torch.tensor(X_train.values, dtype=torch.float32)
y_train_ten = torch.tensor(y_train.values, dtype=torch.float32)

##### Q. 근데 왜 굳이 split을 할까요? cost로 학습하는 거면 모두 넣어도 상관 없지 않을까요?

In [122]:
# shape, ndim, x_train 확인
print(X_train_ten.shape, X_train_ten.ndim, X_train_ten)
print(y_train_ten.shape, y_train_ten.ndim, y_train_ten)

torch.Size([3168, 2]) 2 tensor([[ 9.5000,  0.9927],
        [10.6000,  0.9938],
        [ 8.7000,  1.0001],
        ...,
        [ 9.5000,  0.9958],
        [ 9.6000,  0.9964],
        [12.2000,  0.9908]])
torch.Size([3168]) 1 tensor([5., 6., 5.,  ..., 5., 6., 6.])


2. 학습 준비

In [123]:
# Reset Weight, b value
W = torch.zeros((2,1), requires_grad=True)  
b = torch.zeros(1, requires_grad=True)
print(f'W : {W}, b : {b}')
# requires_grad= : 학습을 통해 값이 계속 변하는 변수임을 의미
# (If autograd should record operations on the returned tensor.)

W : tensor([[0.],
        [0.]], requires_grad=True), b : tensor([0.], requires_grad=True)


In [124]:
# Set SGD : Updata W, b
# - W, b, learning rate
optimizer = optim.SGD([W, b], lr=0.01)

3. 학습 진행

In [125]:
X_train_ten.matmul(W)+b

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)

In [126]:
epochs = 100

for epoch in range(epochs+1):
    print(epoch)
    # 1. function
    y_pred = X_train_ten.matmul(W) + b
    
    # cost
    cost = torch.mean((y_pred - y_train_ten)**2)
    
    # Upgrade H(x) by cost
    optimizer.zero_grad()        # gradient 초기화
    cost.backward()              # cost 미분
    optimizer.step()             # Update W, b
    print(f'W : {W.grad}, b : {b.grad}')
    
    
    # if epoch % 10 == 0:
    #     print(f'Epoch : {epoch},') # W: {W.item()}, b: {b.item()}, Cost: {cost.item()}')

print(W, b, cost, sep='\n\n')

0
W : tensor([[-123.6203],
        [ -11.6253]]), b : tensor([-11.6976])
1
W : tensor([[161.0579],
        [ 14.7972]]), b : tensor([14.8958])
2
W : tensor([[-209.6872],
        [ -19.6135]]), b : tensor([-19.7374])
3
W : tensor([[273.1455],
        [ 25.2008]]), b : tensor([25.3667])
4
W : tensor([[-355.6624],
        [ -33.1621]]), b : tensor([-33.3736])
5
W : tensor([[463.2532],
        [ 42.8459]]), b : tensor([43.1258])
6
W : tensor([[-603.2456],
        [ -56.1415]]), b : tensor([-56.5015])
7
W : tensor([[785.6885],
        [ 72.7730]]), b : tensor([73.2464])
8
W : tensor([[-1023.1630],
        [  -95.1161]]), b : tensor([-95.7281])
9
W : tensor([[1332.5594],
        [ 123.5312]]), b : tensor([124.3327])
10
W : tensor([[-1735.3696],
        [ -161.2196]]), b : tensor([-162.2591])
11
W : tensor([[2260.0881],
        [ 209.6202]]), b : tensor([210.9784])
12
W : tensor([[-2943.3193],
        [ -273.3356]]), b : tensor([-275.1000])
13
W : tensor([[3833.2378],
        [ 355.6328]]), b

W : tensor([[-4797735.0000],
        [ -445303.4062]]), b : tensor([-448182.5625])
41
W : tensor([[6248237.],
        [ 579932.]]), b : tensor([583681.5000])
42
W : tensor([[-8137270.0000],
        [ -755263.5000]]), b : tensor([-760146.6875])
43
W : tensor([[10597415.0000],
        [  983602.2500]]), b : tensor([989961.8750])
44
W : tensor([[-13801336.0000],
        [ -1280975.6250]]), b : tensor([-1289258.])
45
W : tensor([[17973902.0000],
        [ 1668253.1250]]), b : tensor([1679039.2500])
46
W : tensor([[-23407960.],
        [ -2172618.]]), b : tensor([-2186665.])
47
W : tensor([[30484898.0000],
        [ 2829465.5000]]), b : tensor([2847759.5000])
48
W : tensor([[-39701416.0000],
        [ -3684899.5000]]), b : tensor([-3708724.])
49
W : tensor([[51704368.],
        [ 4798958.]]), b : tensor([4829985.])
50
W : tensor([[-67336176.],
        [ -6249828.]]), b : tensor([-6290236.5000])
51
W : tensor([[87693968.],
        [ 8139344.]]), b : tensor([8191969.])
52
W : tensor([[-1.1421

In [127]:
W[0]*1 + W[1]*10

tensor([3.9995e+11], grad_fn=<AddBackward0>)

4. 결과 확인

In [131]:
pred_tensor = torch.tensor([[1.0, 10.0]], dtype=torch.float32)
predic = pred_tensor.matmul(W) + b
predic.item()

419327639552.0